# **Random Forest IT-2**

In [ ]:
# ============================
# NOTEBOOK 1 - MODELO SIMPLE (HYPERPARÁMETROS)
# ============================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1. Cargar datos
df = pd.read_csv("df_preprocesado_final.csv")
print("Shape del dataframe:", df.shape)

# 2. Definir variable objetivo
target = "IARD_global"
y = df[target]

# 3. Construir X excluyendo la variable objetivo
X = df.drop(columns=[target], errors="ignore")

print("Shape X antes de filtrar tipos:", X.shape)

# 4. Quedarse solo con columnas numéricas (incluye dummies 0/1)
X = X.select_dtypes(include=["number"]).copy()

print("Shape X solo numérica:", X.shape)
print("\nEjemplo de columnas numéricas:")
print(X.columns[:20])

# 5. Eliminar filas con NaN en X o en y
mask = X.notna().all(axis=1) & y.notna()
X_clean = X[mask]
y_clean = y[mask]

print("\nFilas usadas tras limpiar NaN:", X_clean.shape[0])

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean,
    test_size=0.3,
    random_state=42
)

print("Tamaño train:", X_train.shape[0], " - Tamaño test:", X_test.shape[0])

# 7. Función de métricas
def resumen_metricas(y_true, y_pred, nombre=""):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"== {nombre} ==")
    print("R²   :", round(r2, 3))
    print("MAE  :", round(mae, 2))
    print("RMSE :", round(rmse, 2))
    print()

# 8. Probar modelos de Random Forest más "simples"
modelos = [
    ("rf_depth5_leaf10",
     RandomForestRegressor(
         n_estimators=300,
         max_depth=5,
         min_samples_leaf=10,
         min_samples_split=20,
         max_features="sqrt",
         random_state=42,
         n_jobs=-1
     )),
    ("rf_depth7_leaf5",
     RandomForestRegressor(
         n_estimators=300,
         max_depth=7,
         min_samples_leaf=5,
         min_samples_split=10,
         max_features="sqrt",
         random_state=42,
         n_jobs=-1
     )),
    ("rf_depthNone_leaf5",
     RandomForestRegressor(
         n_estimators=300,
         max_depth=None,
         min_samples_leaf=5,
         min_samples_split=10,
         max_features="sqrt",
         random_state=42,
         n_jobs=-1
     ))
]

for nombre, modelo_simple in modelos:
    modelo_simple.fit(X_train, y_train)
    y_pred_train = modelo_simple.predict(X_train)
    y_pred_test = modelo_simple.predict(X_test)
    print("====", nombre, "====")
    resumen_metricas(y_train, y_pred_train, "Train")
    resumen_metricas(y_test, y_pred_test, "Test")

# 9. Elegir uno de los modelos (por ejemplo, rf_depth7_leaf5) para ver importancia de variables
rf_final = modelos[1][1]   # modelos[1] = ("rf_depth7_leaf5", modelo)

# Asegurarse de que rf_final está entrenado (por si acaso)
rf_final.fit(X_train, y_train)

importances = rf_final.feature_importances_
feature_names = X_clean.columns

fi_df = pd.DataFrame({
    "variable": feature_names,
    "importancia": importances
}).sort_values("importancia", ascending=False)

print("\nTop 15 variables más importantes (modelo simple):")
print(fi_df.head(15))


Shape del dataframe: (260, 164)
Shape X antes de filtrar tipos: (260, 163)
Shape X solo numérica: (260, 163)

Ejemplo de columnas numéricas:
Index(['2_edad', '12_horas_pedagogicas_semanales',
       '11_años_experiencia_docente', '13_promedio_estudiantes_curso',
       '1º Medio', '2º Medio', '3º Medio', '4º Medio', '7º Básico',
       '8º Básico', '1_genero_Femenino', '1_genero_Masculino',
       '1_genero_Prefiero no decirlo',
       '5_tipo_establecimiento_Administración Delegada',
       '5_tipo_establecimiento_Administración delegada',
       '5_tipo_establecimiento_Corporación',
       '5_tipo_establecimiento_De administración delegada',
       '5_tipo_establecimiento_Municipal',
       '5_tipo_establecimiento_Particular Pagado',
       '5_tipo_establecimiento_Particular Subvencionado'],
      dtype='object')

Filas usadas tras limpiar NaN: 260
Tamaño train: 182  - Tamaño test: 78
==== rf_depth5_leaf10 ====
== Train ==
R²   : 0.085
MAE  : 9.59
RMSE : 11.73

== Test ==
R²   : 0.03